Задание:

Разработать программное обеспечение, реализующее функции генерации секретного и открытого ключей, шифрования и цифровой подписи для алгоритма RSA. Обмен входными и выходными данными должен осуществляться через файлы:
1. открытого ключа;
2. секретного ключа;
3. исходного сообщения;
4. зашифрованного сообщения.
Для повышения скорости шифрования использовать метод последовательного возведения в квадрат и умножения.
Выполнить тестирование разработанного программного обеспечения на 10 наборах тестовых данных.
Длина чисел p и q должна быть не менее 1024 бит.

In [160]:
KEYSIZE = 1024

Генерация больших простых чисел:

In [161]:
import random

def rabin_miller(num):
   s = num - 1
   t = 0
   
   while s % 2 == 0:
      s = s // 2
      t += 1
   for _ in range(5):
      a = random.randrange(2, num - 1)
      v = pow(a, s, num)
      if v != 1:
         i = 0
         while v != (num - 1):
            if i == t - 1:
               return False
            else:
               i = i + 1
               v = (v ** 2) % num
      return True

def is_prime(num):
   if num < 2:
      return False
   lowPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 
   67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 
   157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 
   251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,317, 331, 337, 347, 349, 
   353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 
   457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 
   571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 
   673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 
   797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 
   911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
	
   if num in lowPrimes:
      return True
   for prime in lowPrimes:
      if (num % prime == 0):
         return False
   return rabin_miller(num)

def gcd(a, b):
   while a != 0:
      a, b = b % a, a
   return b

def generate_prime(keysize = 1024):
   while True:
      num = random.randrange(2**(keysize-1), 2**(keysize))
      if is_prime(num):
         return num
      
generate_prime()

98112262212095435653386876060703574151618833785484635146736001318307449700848513585797524807803515547840805898228818012929670579025534901369657595465772591920350913814615175507402382739192257387452843412473904185138031317952191300349142486873756931311372426823017421394734116889230369271504910643874063027473

Теперь, сгенерируем два случайных простых числа p и q, вычислим их произведение, а также функцию Эйлера. В качестве открытой экспоненты используем одно из простых чисел Ферма (17, 257, 65537).

In [162]:
p, q = generate_prime(KEYSIZE), generate_prime(KEYSIZE)

n = p * q
f = (p - 1) * (q - 1)

while True:
    e = random.randrange(2 ** (KEYSIZE - 1), 2 ** KEYSIZE)
    if gcd(e, f) == 1:
        break

Метод find_mod_inverse() вычисляет число d, мультипликативно обратное числу e по модулю n (т.е. удовлетворяет сравнению d*e = 1mod f(n))

In [163]:
def find_mod_inverse(a, m):
   if gcd(a, m) != 1:
      return None
   u1, u2, u3 = 1, 0, a
   v1, v2, v3 = 0, 1, m
   
   while v3 != 0:
      q = u3 // v3
      v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
   return u1 % m

d = find_mod_inverse(e, f)

Таким образом, теперь мы можем составить закрытый и открытый ключи:

In [164]:
public_key = (e, n)
private_key = (d, n)

print(f"Закрытый ключ: {private_key}")
print(f"Открытый ключ: {public_key}")

Закрытый ключ: (193687584421279191871052146476268506340244840616876512166078020979886342060960753583097347776776938617963510301801300885050132143556439081192286034588689840282337775760499864187283961843367676711287881594925530975708545295141458297528763458320520722058244431753800144711351427765047410580023441396935905044006952271054198300853382141817931643714442009677946249994581627693535352336114992305442854072559551059929171860586578290876617415931691720842937146082119141398617525262137382339846134476091520719137911519909433757916245229849912893816635437177238337939048144783826326093428151503866684666890039927537658454467, 1684201418632435326314319671024126508296649759059034681356442955169059076538492298098488407186683603676822657736900362769885289811280675222120977495137421671372191994075763328044561661625865469677834555374657399960097908516637954866490740224346992371519893861368258845761342129804976582380383399656579294953998807449551404936692957142551291214683271579660186444265674

В качестве исходного сообщения будет выступать целое число в интервале от 0 до n - 1.

Для зашифрования используем открытый ключ {e, n}, вычилим крипограмму:

In [165]:
m = random.randint(1, n - 1)

def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

print(f"Исходное сообщение: {m}")

c = fast_mod_exp(m, e, n)

print(f"Криптограмма: {c}")

dc = fast_mod_exp(c, d, n)

print(f"Расшифрованная криптограмма: {dc}")

Исходное сообщение: 4026048807590861258597335694223291235972378036946681328260286658049917637153277821334874336676324239002660074633146056099208353497454965821794317753882010323526077500953929384598916754348247170752870849100206342422528547067581850217837923917483801553746121744980628759033487097646252157522770387766079121960912976578257884193947405040532594033613270803125001889604856412607087905171290159331664360816147084780375937496759149445869580051725194333578185156219341681943654669030453140357927287854037542991949695307090335829824538071658630336976410380129502997655674214416270666143118428595932113059898925310731297824323
Криптограмма: 6329174665177527509865228468096785771673765102036101046039902853508272265941434086891449202501035747079757548682389598859346291878315565959516768379359340741809415914120415701128924108885781783701020670891376003582262445420128519513448319723082953861731120005660755578259909679399847381275988989034478018586778356294225083611650439962166024080105867

Цифровая подпись:

Для создания цифровой подписи s с помощью секретного ключа вычисляют s = m^d mod n. Затем формируют пару {m, s} и отправляют получателю.

In [166]:
s = fast_mod_exp(m, d, n)

signature = (m, s)

print(f"{signature=}")

signature=(4026048807590861258597335694223291235972378036946681328260286658049917637153277821334874336676324239002660074633146056099208353497454965821794317753882010323526077500953929384598916754348247170752870849100206342422528547067581850217837923917483801553746121744980628759033487097646252157522770387766079121960912976578257884193947405040532594033613270803125001889604856412607087905171290159331664360816147084780375937496759149445869580051725194333578185156219341681943654669030453140357927287854037542991949695307090335829824538071658630336976410380129502997655674214416270666143118428595932113059898925310731297824323, 15032727470706916740537552976254454725149024402805757723850017981479043031117508563245150061869289588594686275835301850719954366855862593019899299779227124885432754750046747885389440716512000182448815834879778836081621969260932211336982252194815755432029544150157908393388174969654837806268847280944844002221888643901313312148834405544888881130948135871033660995450413921

Теперь проверим цифровую подпись: вычислим прообраз сообщения из подписи.

In [167]:
ms = fast_mod_exp(s, e, n)

is_valid = m == ms

print(f"{is_valid=}")

is_valid=True
